In [1]:
import os
import soundfile as sf
from pyannote.audio import Pipeline
import warnings
import torchaudio
import torch
from torchaudio.pipelines import WAV2VEC2_BASE
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn as nn
import pandas as pd

In [2]:
#15초 단위 audio chunk 저장하기, 저장경로 지정(음성 입력)
def save_audio_chunks(waveform, sr, chunk_sec=15, save_dir="temp_chunks"):
    os.makedirs(save_dir, exist_ok=True)
    chunk_len = chunk_sec * sr
    total_len = waveform.shape[1]
    
    for i, start in enumerate(range(0, total_len, chunk_len)):
        end = min(start + chunk_len, total_len)
        chunk = waveform[:, start:end]
        out_path = os.path.join(save_dir, f"chunk_{i:03d}.wav")
        sf.write(out_path, chunk.T, sr)  # chunk.T to shape (samples, channels)
        print(f"Saved: {out_path}")

In [ ]:
# diarization 실행 코드
HF_TOKEN = 'hf'

#pipeline 로드
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=HF_TOKEN
)
chunk_dir =  "./temp_chunks"

#diarization 수행
for fname in sorted(os.listdir(chunk_dir)):
    if not fname.endswith(".wav"):
        continue
    file_path = os.path.join(chunk_dir, fname)
    print(f"▶️ Diarizing: {file_path}")

    diarization = pipeline(file_path)

    for turn, _, speaker in diarization.itertracks(yield_label=True):
        print(f"{fname}: {turn.start:.1f}–{turn.end:.1f}s speaker_{speaker}")

C:\Users\asia\.conda\envs\py312tf9\Lib\inspect.py:1007: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):


▶️ Diarizing: ./temp_chunks\test.wav


C:\Users\asia\.conda\envs\py312tf9\Lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\ReduceOps.cpp:1839.)
  std = sequences.std(dim=-1, correction=1)


test.wav: 0.2–0.5s speaker_SPEAKER_01
test.wav: 1.2–1.7s speaker_SPEAKER_01
test.wav: 2.3–10.5s speaker_SPEAKER_00
test.wav: 11.3–11.7s speaker_SPEAKER_02


In [4]:
if os.path.exists("user_embedding.npy"):
    os.remove("user_embedding.npy")
    print("✅ user_embedding.npy 삭제 완료")
else:
    print("❌ 파일이 존재하지 않습니다.")

❌ 파일이 존재하지 않습니다.


In [7]:
#사용자 임베딩 저장

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

bundle = torchaudio.pipelines.WAV2VEC2_BASE
model = bundle.get_model().to(device).eval()

def extract_wav2vec2_embedding(path):
    waveform, sr = torchaudio.load(path)
    
    # 💡 [중요] 다채널인 경우 평균을 내어 mono로 변환
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)  # → (1, time)

    if sr != bundle.sample_rate:
        waveform = torchaudio.functional.resample(waveform, sr, bundle.sample_rate)

    waveform = waveform.to(device)

    with torch.inference_mode():
        out = model(waveform)  # (batch=1, time, 768)

    emb = out[0].mean(dim=1)  # → (1, 768)

    return emb.cpu().numpy()  # → (1, 768)

In [9]:
user_embedding = extract_wav2vec2_embedding(user_path)
print("✅ user_embedding.shape:", user_embedding.shape)  # 반드시 (1, 768)이어야 함
np.save("user_embedding.npy", user_embedding)

NameError: name 'user_path' is not defined

In [26]:
chunk_path = "./temp_chunks/test.wav"

diarization_result = pipeline(chunk_path)
output_dir = './diarized_chunks'
os.makedirs(output_dir, exist_ok=True)

waveform, sr = torchaudio.load(chunk_path)
waveform = waveform.squeeze().numpy()

for i, (segment, _ , speaker) in enumerate(diarization_result.itertracks(yield_label=True)):
    start_sample = int(segment.start * sr)
    end_sample = int(segment.end * sr)
    segment_wave = waveform[start_sample:end_sample]
    out_path = os.path.join(output_dir, f"{os.path.basename(chunk_path).replace('.wav','')}_spk{speaker}_{i:03d}.wav")
    sf.write(out_path, segment_wave, sr)
    print(f"Saved: {out_path}")

Saved: ./diarized_chunks\test_spkSPEAKER_01_000.wav
Saved: ./diarized_chunks\test_spkSPEAKER_01_001.wav
Saved: ./diarized_chunks\test_spkSPEAKER_00_002.wav
Saved: ./diarized_chunks\test_spkSPEAKER_02_003.wav


In [41]:
print(bundle)

Wav2Vec2Bundle(_path='wav2vec2_fairseq_base_ls960.pth', _params={'extractor_mode': 'group_norm', 'extractor_conv_layer_config': [(512, 10, 5), (512, 3, 2), (512, 3, 2), (512, 3, 2), (512, 3, 2), (512, 2, 2), (512, 2, 2)], 'extractor_conv_bias': False, 'encoder_embed_dim': 768, 'encoder_projection_dropout': 0.1, 'encoder_pos_conv_kernel': 128, 'encoder_pos_conv_groups': 16, 'encoder_num_layers': 12, 'encoder_num_heads': 12, 'encoder_attention_dropout': 0.1, 'encoder_ff_interm_features': 3072, 'encoder_ff_interm_dropout': 0.0, 'encoder_dropout': 0.1, 'encoder_layer_norm_first': False, 'encoder_layer_drop': 0.05, 'aux_num_out': None}, _sample_rate=16000, _normalize_waveform=False, _model_type='Wav2Vec2')


In [53]:
# 화자 식별 코드
#사용자 입데이 불러오기
user_embedding = np.load('user_embedding.npy')

#diarized_cunks 폴더에서 segment별 음성 비교
diarized_dir = './diarized_chunks'
user_segments = []
similarity_threshold = 0.8 # 유사성 임계값
for fname in os.listdir(diarized_dir):
    if not fname.endswith(".wav"):
        continue
    path = os.path.join(diarized_dir, fname)
    segment_embedding = extract_wav2vec2_embedding(path)

    sim = cosine_similarity(user_embedding, segment_embedding)[0][0]

    if sim >= similarity_threshold:
        print(f"✅ 사용자 발화: {fname} (similarity: {sim:.3f})")
        user_segments.append(fname)
    else:
        print(f"❌ 타인 발화: {fname} (similarity: {sim:.3f})")

✅ 사용자 발화: test_spkSPEAKER_00_002.wav (similarity: 0.812)
❌ 타인 발화: test_spkSPEAKER_01_000.wav (similarity: 0.602)
❌ 타인 발화: test_spkSPEAKER_01_001.wav (similarity: 0.557)
❌ 타인 발화: test_spkSPEAKER_02_003.wav (similarity: 0.710)


In [56]:
# 사용자 발화 병합 및 저장
merged_waveform = []
sr = bundle.sample_rate  # Wav2Vec2 모델 샘플레이트

for fname in sorted(user_segments):  # 시간 순서를 위해 정렬
    path = os.path.join(diarized_dir, fname)
    waveform, _ = torchaudio.load(path)
    
    # 다채널이면 mono로 변환
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    
    merged_waveform.append(waveform)

# 병합
merged_waveform = torch.cat(merged_waveform, dim=1)  # (1, total_samples)

# 저장
output_path = "./user_segments_merged.wav"
sf.write(output_path, merged_waveform.squeeze().numpy(), sr)
print(f"✅ 사용자 음성 segment 병합 완료: {output_path}")

✅ 사용자 음성 segment 병합 완료: ./user_segments_merged.wav


In [10]:
# 임시 감정 분류기
class DummyEmotionClassifier(nn.Module):
    def __init__(self, input_dim=768, num_classes=5):
        super().__init__()
        self.linear = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        return self.linear(x)

# 감정 라벨 정의 (실제 모델에 맞춰 수정)
emotion_labels = ['anger', 'happiness', 'neutral', 'sadness']

# 모델 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
emotion_model = EmotionMLP().to(device)
emotion_model.load_state_dict(torch.load("emotion_classifier.pth", map_location=device))
emotion_model.eval()

# 오디오에서 임베딩 추출
merged_path = "./user_segments_merged.wav"
embedding = extract_wav2vec2_embedding(merged_path)  # (1, 768)

# 감정 예측
with torch.no_grad():
    logits = emotion_model(torch.from_numpy(embedding).to(device).float())
    pred = torch.argmax(logits, dim=1).item()
    emotion = emotion_labels[pred]

# 결과 저장
df = pd.DataFrame([{
    "timestamp": pd.Timestamp.now(),
    "file": os.path.basename(merged_path),
    "predicted_emotion": emotion
}])
df.to_csv("emotion_log.csv", index=False)

print(f"🎯 예측된 감정: {emotion}")
print("📄 emotion_log.csv에 저장 완료")

NameError: name 'EmotionMLP' is not defined